In [1]:
import jax
import jax.numpy as jnp
from jax import random, grad, vmap, jit

from ngrad.models import init_params, mlp
from ngrad.domains import Square, SquareBoundary
from ngrad.integrators import DeterministicIntegrator
from ngrad.utility import laplace, grid_line_search_factory
from ngrad.inner import model_laplace, model_identity
from ngrad.gram import gram_factory, nat_grad_factory

In [2]:
jax.config.update("jax_enable_x64", True)

In [3]:
Square(1.)

In [4]:
intervals = [[0., 1.], [0., 1.], [0., 1.]] 

In [5]:
a = jnp.array(intervals)

In [9]:
len(jnp.shape(a))

2

In [10]:
seed = 1
sizes = [10, 32, 32, 1]

In [11]:
params = init_params(sizes, jax.random.PRNGKey(seed))

In [29]:
import math
a = 1
N = 0.1

In [30]:
M = max(math.ceil(a) * N, 2)

In [31]:
M

2

In [34]:
math.ceil(1.1)

2

In [40]:
for i in range(1,2):
    print(i)

1


In [61]:
from ngrad.domains import Square, Hyperrectangle

In [57]:
S = Square(1.5)

In [58]:
a = S.deterministic_integration_points(32)

In [59]:
a.shape

(3844, 2)

In [68]:
H = Hyperrectangle( [[0., 1.], [0., 3.], [0., 2.]] )

In [69]:
key = jax.random.PRNGKey(0)
b = H.random_integration_points(key, 100)

In [70]:
b.shape

(100, 3)

In [ ]:
slice()

slice(None, 10, None)

In [82]:
[0, 1, 2, 3][slice(0,3)]

[0, 1, 2]